## **PVSyst Report Scraper:**

Search for specific measures in PVSyst Report PDF Files;

### Import

In [ ]:
import re
from pathlib import Path
from PyPDF2 import PdfReader
import pandas as pd
import os
from tkinter.filedialog import askdirectory

### Set Report Directory
Define the directory where the PDF files are located.

In [ ]:
# By UI selection
your_path = askdirectory(title='Open Directory')
directory = Path(your_path)
directory

In [ ]:
# By pasting the path
your_path = r'\\location' # Replace `\\location` with the right path
directory = Path(your_path)
directory

### Check for files inside direcotry

In [ ]:
files_list = []
for filename in directory.glob('*.pdf'):
    # open the PDF file
    files_list.append(str(filename).split('\\')[-1])

print('Count of files:',len(files_list))
files_list

### Scraper

In [ ]:
# Define the pattern to search for
pattern = r'Specific production (\d+) kWh/kWp/year'

files =[]
spec_prod = []
errors = []
# Loop through all the PDF files in the directory
for full_file_path in directory.glob('*.pdf'):
    
    with open(full_file_path, 'rb') as file: # Open the PDF file
        
        file_name = str(full_file_path).split('\\')[-1]
        pdf_reader = PdfReader(file) # Instance the file as a PdfReader object 
        text = pdf_reader.pages[1].extract_text() + pdf_reader.pages[2].extract_text() # and extract the text from pages 1 and 2
        
        match = re.search(pattern, text) # search for the pattern in the text
        
        if match:
            # append the file name and the matched PE value to the lists
            files.append(file_name)
            spec_prod.append(match.group(1))
        else:
            errors.append(file_name)

result_df = pd.DataFrame(data=zip(files, spec_prod),columns='PV_Plant Spec_Prod'.split())


In [ ]:
result_df

In [ ]:
errors

In [ ]:
def open_errors(error_list, dir, limit=10) -> None:
    '''
    Open externaly the files that couldn't be scraped by the code.
    Limit of openned files defined by arg 'limit' (10 by default)
    '''
    for i, file_name in enumerate(error_list):
        if not i >= limit:
            os.startfile(Path(str(dir)+'\\'+file_name))
    return

open_errors(errors, directory)
